# Results: Manual vs Automated Feature Engineering

In this notebook, we will compare the manual and automated feature engineering approaches for the Kaggle Home Credit Default Risk competition. The comparison will be done in terms of time (how long it took to complete) and the performance (the score in cross validation and when submitted to the Kaggle leaderboard.) 

## Manual Results

In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [4]:
manual = pd.read_csv('../input/results/random_search_domain_manual_features_fast.csv').sort_values('score', ascending = False)
manual.head()

,score,hyperparameters,iteration,time
125,0.780678,"{'verbose': 1, 'is_unbalance': True, 'boosting...",125,384.011306
109,0.780417,"{'verbose': 1, 'is_unbalance': True, 'boosting...",109,370.445021
9,0.780216,"{'verbose': 1, 'is_unbalance': True, 'boosting...",9,137.456086
41,0.780154,"{'verbose': 1, 'is_unbalance': True, 'boosting...",41,134.778583
97,0.780134,"{'verbose': 1, 'is_unbalance': True, 'boosting...",97,169.808482
